In [1]:
import sys
import os
import cv2
import numpy as np

import json
# from scipy.optimize import linear_sum_assignment
from ortools.sat.python import cp_model

In [2]:
sys.path.append("../0.mc_utils/")

In [3]:
from common.tools_cv import draw_kp

In [4]:
def load_op_result(json_fpath):
    with open(json_fpath,'r') as f:
        json_data = json.load(f)
        kp2ds_ = json_data['people']
        if len(kp2ds_) == 0:
            return None
        people_num = len(kp2ds_)
        kp2ds = []
        for i in range(len(json_data['people'])):
            kp2ds_ = np.array(json_data['people'][i]['pose_keypoints_2d']).reshape([25,3])
            kp2ds.append(kp2ds_)
        return kp2ds
    
def load_op_folder(json_folder):
    kp_list = []
    for root, dirs, files in os.walk(json_folder):  
        for file in files:  
            if os.path.splitext(file)[1] == '.json':  
                json_fpath = os.path.join(root, file)
                temp = load_op_result(json_fpath)
                #print(temp.shape)
                kp_list.append(temp)
    return kp_list

In [5]:
a1 = load_op_folder("../../yunchong_reid_data/v1")
a2 = load_op_folder("../../yunchong_reid_data/v2")
a3 = load_op_folder("../../yunchong_reid_data/v3")

In [6]:
class singlePerson(object):
    def __init__(self,num_view,pid = 0):
        self.pid = pid
        self.num_view = num_view
        #last_pose is used to storage the last time person poses in every views in turn,
        #if one element is [],that means there is no pose in that view
        self.last_view_pose = []
        for i in range(self.num_view):
            self.last_view_pose.append([])

class PersonReID(object):
    def __init__(self,num_view = 1):
        self.num_view = num_view
        self.F_list = np.zeros([self.num_view,self.num_view,3,3])
        self.current_t = 0
        self.current_num_person = 0
        self.person = []
        
        
    def distributeLoss_(self,c_person,l_person):
        loss = 999.0
        for one in c_person:
            if len(l_person.last_view_pose[one[1]]) == 0:
                continue
            l_one = l_person.last_view_pose[one[1]][-1]
            temploss = pose_loss(one[0],l_one[0])
            if temploss < loss:
                loss = temploss
        return loss
    
    def initF(self,person_array):
        person = []
        for i in range(len(person_array)):
            person.append(np.concatenate(person_array[i]))
        for i in range(len(person_array) - 1):
            for j in range(i+1,len(person_array)):
                F,_ = solverF(person[i],person[j])
                if _ == True:
                    self.F_list[i,j] = F
                    
      
        
        
    def update(self,current_pose,test = 0):    #t means the t frame,current_pose means all poses in the t frame in different views in turn
        self.current_t += 1
        current_idx = []
        #use ortools to mate all poses in one frame 
        #initial ortools
        model = cp_model.CpModel()
        y = []
        num_vals = 0
        num_per_scene = []
        
        for one_scene_pose in current_pose:
            num_vals += len(one_scene_pose)
            num_per_scene.append(len(one_scene_pose))  
        for i in range(num_vals):
            y.append([])
            for j in range(num_vals):
                y[i].append(model.NewIntVar(0,1,'t['+str(i)+']['+str(j)+']'))
        for i in range(num_vals-2):
            for j in range(i,num_vals-1):
                for k in range(j,num_vals):
                    model.Add(y[i][j]+y[j][k] <= 1 + y[i][k])
        
        for i in range(num_vals):
            current_j = 0
            for num in num_per_scene:
                temp = 0
                for j in range(num):
                    temp += y[i][j + current_j]
                current_j += num
                model.Add(temp <= 1)
        
        for i in range(num_vals):
            current_j = 0
            for num in num_per_scene:
                temp = 0
                for j in range(num):
                    temp += y[j + current_j][i]
                current_j += num
                model.Add(temp <= 1)

        detections = []
        for i in range(len(current_pose)):
            for pose in current_pose[i]:
                detections.append([pose,i])
                
        A = np.ones([num_vals,num_vals])*-1
        for i in range(len(detections)-1):
            for j in range(i+1,len(detections)):
                A[i,j] = 100*affinity(detections[i][0],detections[j][0],self.F_list[detections[i][1],detections[j][1]])
                if A[i,j] < 0:
                    A[i,j] = -1
                    A[j,i] = -1
        sum_ = 0
        if test == 429:
            print("a",A)
        for i in range(num_vals):
            for j in range(num_vals):
                sum_ += int(A[i,j])*y[i][j]
        model.Maximize(sum_)
        solver = cp_model.CpSolver()
        status = solver.Solve(model)
        if test == 429:
            tr = []
            for i in range(len(detections)):
                tra = []
                for j in range(len(detections)):
                    tra.append(solver.Value(y[i][j]))
                tr.append(tra)
        #mate poses in current frame
        current_person = []
        temp_mate = np.ones([1,len(detections)])*-1
        for i in range(len(detections)):
            if temp_mate[0,i] == -1:
                current_person.append([detections[i]])
                temp_mate[0,i] = 0
                for j in range(i+1,len(detections)):
                    if solver.Value(y[i][j]) == 1:
                        current_person[-1].append(detections[j])
                        temp_mate[0,j] = 0

        #---------------------------------------------------------------------
        #mate current person into last person
        temp_clist = list(range(0,len(self.person)))
        residue_person = []
        for current_single_person in current_person:
            temp_c = -1
            loss = 99.0
            for i in range(len(temp_clist)):
                temploss = self.distributeLoss_(current_single_person,self.person[temp_clist[i]])
                #print(temploss)
                if temploss < loss:
                    temp_c = i
                    loss = temploss
            if temp_c != -1:
                if test == 429:
                    print(loss)
                current_idx.append([current_single_person,self.person[temp_clist[temp_c]].pid])
                for one_camera_new_person in current_single_person:
                    self.person[temp_clist[temp_c]].last_view_pose[one_camera_new_person[1]].append([one_camera_new_person[0],self.current_t])
                            
                temp_clist.pop(temp_c)
                #print("one person")
            else:
                residue_person.append(current_single_person)
        for new_person in residue_person:
            current_idx.append([new_person,self.current_num_person])
            temp = singlePerson(self.num_view,self.current_num_person)
            for one_camera_new_person in new_person:
                temp.last_view_pose[one_camera_new_person[1]].append([one_camera_new_person[0],self.current_t])
            self.person.append(temp)
            self.current_num_person += 1
        
        
        result = []
        for i in range(self.num_view):
            result.append([])
            
        for i in range(len(current_idx)):
            for j in range(len(current_idx[i][0])):
                result[current_idx[i][0][j][1]].append([current_idx[i][0][j][0],current_idx[i][1]])
                
        
        return result
                
            
        

In [7]:
#Calculate the loss between two poses by calculate the L2 norm of the joints differ
#input two 2D pose from the same view but different  
def pose_loss(ja,jb):
    #input ja,jb as np.array([J,2])
    assert ja.shape[0] == jb.shape[0]
    J = ja.shape[0]
    real_J = 0
    sum_loss = 0
    max_loss = 0
    max_loss2 = -1
    
    for j in range(J):
        if ja[j].any() != np.zeros([1,2]).any() and jb[j].any() != np.zeros([1,2]).any():    #You should select the joints that are discriminated in both poses
            real_J += 1
            if max_loss < np.linalg.norm(ja[j] - jb[j]):
                max_loss2 = max_loss
                max_loss = np.linalg.norm(ja[j] - jb[j])
            
            sum_loss += np.linalg.norm(ja[j] - jb[j])
    
    if real_J <= 3:
        return 999.0
    norm_pose_loss = (sum_loss)/(real_J)
    return norm_pose_loss

def solverF(ja,jb):
    assert ja.shape[0] == jb.shape[0]
    real_J_list = []
    J = ja.shape[0]

    for j in range(J):
        if ja[j].any() != np.zeros([1,2]).any() and jb[j].any() != np.zeros([1,2]).any():
            real_J_list.append(j)
            
    #Need at least eight_point to solver the matrix
    n = len(real_J_list)
    if n < 8:
        return np.zeros([3,3]),False
    
    a1 = ja[real_J_list]
    b1 = jb[real_J_list]

    F2,w = cv2.findFundamentalMat(a1,b1)
    
    return F2,True

def affinity(ja,jb,F):     
    assert ja.shape[0] == jb.shape[0]
    
    if F.all() == np.zeros([3,3]).all():
        #print("F zero same view")
        return -1
    J = ja.shape[0]
    real_J_list = []
    for j in range(J):
        if ja[j].any() != np.zeros([1,2]).any() and jb[j].any() != np.zeros([1,2]).any():
            real_J_list.append(j)
    
    sum_aff = 0.0
    n = len(real_J_list)
    if n < 8:
        return -1
    for i in range(n):
        ji = real_J_list[i]
        x1 = np.array([[ja[ji,0]],[ja[ji,1]],[1]])
        x2 = np.array([[jb[ji,0]],[jb[ji,1]],[1]])
        
        F1 = np.dot(x2.T,F)
        F2 = np.dot(F,x1)
        sum_aff += 1.0 - (abs(np.dot(F1,x1))/np.linalg.norm(F1[0:2]) + abs(np.dot(x2.T,F2))/np.linalg.norm(F2[0:2]))/200.0
    norm_aff = sum_aff/n
    return norm_aff[0][0]

In [8]:
t = PersonReID(3)
aa1 = []
aa2 = []
aa3 = []
for i in range(20,25):
    for j in range(1):
        aa1.append(a1[i][j][:,:2])
    
    for j in range(1,2):
        aa2.append(a2[i][j][:,:2])
        
#     for j in range(1):
#         aa3.append(a3[i][j][:,:2])
# t.initF([aa1,aa2,aa3])
t.initF([aa1,aa2])

for k in range(10,1000):
    aaa1 = []
    for i in range(len(a1[k])):
        aaa1.append(a1[k][i][:,:2])
    aaa2 = []
    for i in range(len(a2[k])):
        aaa2.append(a2[k][i][:,:2])
    aaa3 = []
    for i in range(len(a3[k])):
        aaa3.append(a3[k][i][:,:2])
        
#     att = [aaa1,aaa2,aaa3]
    att = [aaa1,aaa2]
    res = t.update(att,k)
    im = np.zeros([750,1024])
    aaaaa = 0
    temp = []
    for i in range(len(res)):
        for j in range(len(res[i])):
            per = res[i][j]
            if per[1] == 0:
                aaaaa += 1
                im = draw_kp(im,per[0][np.newaxis])
                temp.append([i,j])
    cv2.imshow('Image1',im)
    if cv2.waitKey(1) & 0xFF==ord('q'):
        break
cv2.destroyWindow("Image1")     